# MathAI

<img src="https://github.com/droyti46/aiijc2025/blob/main/img/snr-logo.png?raw=true" width=600px>

## Архитектура решения

### Общая идея

Пайплайн берёт задачи на русском языке, автоматически подготавливает их для обработки моделью (очищает, расставляет математические символы, переводит на английский), затем решает задачи с помощью LLM, извлекает ответы, приводит их к числовой форме и в конце выбирает наиболее правдоподобный результат.

<img src="https://github.com/droyti46/aiijc2025/blob/main/img/stage1/architecture.png?raw=true" width=800px>

#### 1. Загрузка данных

* Загружается файл `test_private.csv` с задачами.
* Каждая задача хранится в колонке `task`.

#### 2. Предобработка текста

1. **`auto_wrap_math`**
   Автоматически оборачивает математические выражения в `$...$`, чтобы они воспринимались как формулы.

2. **`strip_empty_dollars`**
   Убирает пустые `$`, которые могли появиться рядом с русскими словами и не несут смысла.

3. **Замена запятой в числах на точку**
   Например: `3,14` → `3.14`.

После этого появляется новая колонка `need_task` — очищенный вариант задачи, готовый для перевода.

#### 3. Перевод задач на английский

* Используется LLM (модель **Qwen3-4B-Thinking**).
* В prompt задаётся строгая инструкция: **перевести на английский, сохранив все формулы и нотацию без изменений**.
* Полученный результат записывается в колонку `eng_task`.
  Это нужно, чтобы модель-решатель работала на английских формулировках (так точность выше).

#### 4. Решение задач

* Для каждой задачи формируется запрос:
  *«Solve task, put answer inside `\boxed{...}`...»*
* Модель получает задачу на английском и выдаёт несколько (n=8) ответов.
* Все ответы сохраняются в колонках `response_0`, `response_1`, ...

#### 5. Извлечение и парсинг ответа

Используются функции:

1. **`extract_boxed`**
   Достаёт содержимое из `\boxed{...}`

2. **`parse_answer`**
   Приводит ответ к числовой форме:

   * заменяет `π` на `3.1415...`;
   * вычисляет `e^{\sin 2}`, `e^{\cos π}` и т. п.;
   * преобразует дроби `\frac{a}{b}` в `a/b`;
   * поддерживает `\infty`.

Таким образом, разные варианты ответа сводятся к одной числовой записи.

#### 6. Постобработка и выбор финального ответа

* Для каждой задачи собираются все распарсенные ответы (`response_0`, `response_1`, …).
* Считается частота каждого уникального значения.
* Выбирается наиболее часто встречающийся ответ (majority vote).
* Финальный результат записывается в CSV (`final.csv`) в формате `[ответ]`.


## Импорты, загрузка и настройка

In [ ]:
!pip install --upgrade vllm

In [ ]:
# Базовые библиотеки
import pandas as pd
import numpy as np
import re
import math
import gc
from collections import Counter
from tqdm import tqdm

# Библиотеки для работы с LLM
import torch
from vllm import LLM, SamplingParams

In [ ]:
# test_private.csv должен находиться в директории с блокнотом
df = pd.read_csv('test_private.csv')
df.head()

## Вспомогательные функции для предобработки

In [ ]:
def auto_wrap_math(text: str) -> str:
    """
    Автоматически оборачивает математические выражения в $ ... $

    Args:
        text (str): Исходный текст

    Returns:
        str: Текст с обернутыми математическими выражениями
    """
    # Разбиваем на сегменты: уже обернутые $...$ и остальное
    parts = re.split(r"(\$.*?\$)", text)
    wrapped = []

    # Регулярка для кластеров: команды, буквы/цифры, операторы, %, скобки
    cluster_re = re.compile(r"(?:\\[A-Za-z]+|[A-Za-z0-9]+|[+\-*/^_=<>|%\[\](){}])+")

    for part in parts:
        # Если сегмент уже в $...$, оставляем без изменений
        if part.startswith('$') and part.endswith('$'):
            wrapped.append(part)
            continue

        # Функция обертки для кластера
        def wrap(m):
            s = m.group(0)
            # Не оборачиваем одиночные круглые скобки
            if s in ('(', ')'):
                return s
            # Если уже содержит $, не трогаем
            if '$' in s:
                return s
            # Условие обертки для математических выражений
            if re.search(r"\\|\d|[+\-*/_=<>|%\[\]{}]", s) or re.fullmatch(r"[A-Za-z]+", s):
                return f'${s}$'
            return s

        # Применяем кластерную обертку
        new_seg = cluster_re.sub(wrap, part)
        wrapped.append(new_seg)

    result = ''.join(wrapped)
    # Убираем пустые обертки вроде '$$' или '$    $'
    result = re.sub(r"\$\s*\$", "$", result)
    return result

In [ ]:
def is_russian_letter(char: str) -> bool:
    """
    Проверяет, является ли символ русской буквой

    Args:
        char (str): Проверяемый символ

    Returns:
        bool: True если русская буква
    """
    return bool(re.match(r'^[а-яёА-ЯЁ]$', char))

In [ ]:
def strip_empty_dollars(s: str) -> str:
    """
    Удаляет пустые $ символы, которые не содержат математики
    рядом с русскими буквами

    Args:
        s (str): Входная строка

    Returns:
        str: Очищенная строка
    """
    is_first = True
    have_ru = False
    out = ''
    cur = ''

    for ind, i in enumerate(s):
        cur += i
        if i == '$':
            # Есть ли русские буквы после?
            have_ru_after = False
            for j in s[ind + 1:]:
                if j == '$':
                    break
                if is_russian_letter(j):
                    have_ru_after = True
                    break

            if is_first:
                is_first = False
                have_ru = False
                continue

            if not have_ru and not have_ru_after and ind != s.rfind('$'):
                cur = cur[:-1]

            out += cur
            cur = ''
            have_ru = False

        if is_russian_letter(i):
            have_ru = True

    return out + cur

## Применение предобработки

In [ ]:
# Применяем предобработку
for i in range(len(df)):
    original = df.at[i, 'task']
    original = original.replace('$', '')
    df.at[i, 'task_with_dollars'] = auto_wrap_math(original)


In [ ]:
df['need_task'] = df['task_with_dollars'].apply(lambda x: strip_empty_dollars(x))
pattern = r'(?<=\d),(?=\d)'
df['need_task'] = df['need_task'].str.replace(pattern, '.', regex=True)
df['need_task']

## Функции для парсинга математических ответов

In [ ]:
def extract_boxed(text: str) -> str | None:
    """
    Извлекает содержимое последнего \\boxed{…} с поддержкой вложенных скобок

    Args:
        text (str): Текст для поиска

    Returns:
        str | None: Содержимое boxed или None если не найдено
    """
    start = text.find(r'\boxed{')
    if start == -1:
        return None
    i = start + len(r'\boxed{')
    depth = 1
    buf = []
    while i < len(text) and depth > 0:
        c = text[i]
        if c == '{':
            depth += 1
        elif c == '}':
            depth -= 1
            if depth == 0:
                break
        buf.append(c)
        i += 1
    return ''.join(buf).strip() if depth == 0 else None

In [ ]:
def parse_answer(text: str) -> str:
    """
    Парсит математический ответ из текста и вычисляет численные значения

    Обрабатывает:
    - e^{\\cot3}, e^{\\sin 2}, e^{\\cos π}
    - \\frac{a}{b}, \\dfrac{a}{b}
    - 2π → 6.283...

    Args:
        text (str): Текст с ответом

    Returns:
        str: Численное значение ответа
    """
    boxed = extract_boxed(text)
    if not boxed:
        return None

    ans = boxed.replace(" ", "")

    # 1) Обработка π
    ans = re.sub(
        r"([-+]?[0-9]*\.?[0-9]+)?(?:\\pi|π)",
        lambda m: str(float(m.group(1) or 1) * math.pi),
        ans
    )

    # 2) e^{...} с тригонометрией (включая \cot3 без скобок)
    m_e = re.fullmatch(r"e\^\{(.+?)\}", ans)
    if m_e:
        inner = m_e.group(1)

        def eval_trig(expr: str) -> float:
            """Вычисляет тригонометрическое выражение в радианах."""
            expr = expr.strip()

            # Число
            try:
                return float(expr)
            except ValueError:
                pass

            # π
            if expr in [r'\pi', 'π']:
                return math.pi

            # Дробь
            if '/' in expr:
                try:
                    a, b = expr.split('/')
                    return float(a) / float(b)
                except:
                    pass

            # Тригонометрические функции (с поддержкой \sin3 и \sin{3})
            patterns = [
                (r"\\sin\{?(-?\d*\.?\d+)\}?", math.sin),
                (r"\\cos\{?(-?\d*\.?\d+)\}?", math.cos),
                (r"\\tan\{?(-?\d*\.?\d+)\}?", math.tan),
                (r"\\cot\{?(-?\d*\.?\d+)\}?", lambda x: 1/math.tan(x)),
                (r"\\sec\{?(-?\d*\.?\d+)\}?", lambda x: 1/math.cos(x)),
                (r"\\csc\{?(-?\d*\.?\d+)\}?", lambda x: 1/math.sin(x)),
            ]

            for pattern, func in patterns:
                match = re.fullmatch(pattern, expr)
                if match:
                    arg = float(match.group(1))
                    return func(arg)

            raise ValueError(f"Неизвестное выражение: {expr}")

        try:
            # Вычисляем показатель степени
            exp_value = eval_trig(inner)
            return str(math.e ** exp_value)
        except:
            # Fallback для простых случаев
            try:
                return str(math.e ** float(inner))
            except:
                return ans

    # 3) Дроби: \frac{a}{b} и \dfrac{a}{b}
    ans = re.sub(
        r"\\[d]?frac\{\s*(-?\d+)\s*\}\{\s*(-?\d+)\s*\}",
        lambda m: f"{m.group(1)}/{m.group(2)}",
        ans
    )

    # 4) Численные значения
    if re.fullmatch(r"[-+]?\d+(?:\.\d+)?(?:e[-+]?\d+)?", ans, re.IGNORECASE):
        return ans

    if ans == r'\infty':
      return "inf"

    return ans

## Перевод задач на английский

In [ ]:
# Используем Qwen3-4B-Thinking для перевода и решения задач
# Эта модель показывает хорошие результаты на математических задачах и хорошие результаты в переводе
model_path = "Qwen/Qwen3-4B-Thinking-2507"
llm = LLM(
    model=model_path,
    trust_remote_code=True,
    gpu_memory_utilization=0.95,
    max_model_len=45000
)

In [ ]:
key_prompt = '''You are given a problem statement in Russian. Your task is to translate it into English with maximum precision, keeping all mathematical formulas and notation unchanged. The translation must preserve the exact meaning so that the problem remains solvable.

Analyze the Russian text, resolve ambiguities, decide on the best English wording, ensure mathematical correctness

Now output only the translated problem statement in English, with no commentary, explanation, or extra text.'''

### Формирование запросов

In [ ]:
from tqdm import tqdm
conversations = []
all_batch_idxs = []

for idx in tqdm(range(len(df))):
    task = df.loc[idx, 'need_task']

    conversations.append([
        {"role": "user", "content": key_prompt + '\nTask:\n' + task}
    ])
    all_batch_idxs.append(idx)

### Запуск языковой модели

In [ ]:
sampling_params = SamplingParams(max_tokens=10000, temperature=0.0)
outputs = llm.chat(
    conversations,
    sampling_params=sampling_params,
    use_tqdm=True,
)

In [ ]:
from collections import Counter
outputs2 = outputs.copy()

nothing = []

for idx, output in tqdm(zip(all_batch_idxs, outputs), total=len(outputs)):
    for i, candidate in enumerate(output.outputs):
        response = candidate.text.strip()
        try:
            df.loc[idx, 'eng'] = response
        except Exception as e:
            print(f"Failed to parse answer for idx={idx}, candidate={i}: {e}")
            nothing.append([idx, response])

In [ ]:
# Извлекаем текст после последнего 'think' + 8 символов
# Это специфика формата ответов модели Qwen3-4B-Thinking
# Также удаляем артефакт "Translate from English to Russian:" если он есть
df['eng_task'] = df['eng'].map(lambda x: x[x.rfind('think') + 8:].replace('Translate from English to Russian:', ''))

## Решение математических задач

In [ ]:
key_prompt = 'Solve task, make sure you put answer inside \\boxed{...}. Please note that in tasks where there are several answers, they must be listed using ; (or as requested in the task). Also keep in mind that if you need to find the eigenvalues of a matrix, the answer is also through ;, but all values must be unique.'

### Формирование запросов

In [ ]:
from tqdm import tqdm
conversations = []
all_batch_idxs = []

for idx in tqdm(range(len(df))):
    task = df.loc[idx, 'eng_task']
    conversations.append([
        {"role": "user", "content": key_prompt + '\nTask:\n' + task}
    ])
    all_batch_idxs.append(idx)

### Запуск языковой модели

In [ ]:
sampling_params = SamplingParams(max_tokens=45000, n=8, temperature=0.3)

In [ ]:
outputs = llm.chat(
    conversations,
    sampling_params=sampling_params,
    use_tqdm=True,
)

In [ ]:
outputs2 = outputs.copy()

nothing = []

for idx, output in tqdm(zip(all_batch_idxs, outputs), total=len(outputs)):
    for i, candidate in enumerate(output.outputs):
        response = candidate.text.strip()
        try:
            df.at[idx, f'response_{i}'] = response
            print(idx, parse_answer(df.at[idx, f'response_{i}']))
        except Exception as e:
            print(f"Failed to parse answer for idx={idx}, candidate={i}: {e}")
            nothing.append([idx, response])

## Постобработка и выбор финальных ответов


In [ ]:
import pandas as pd
import re
df = pd.read_csv('greedyQwen3_4B.csv')

In [ ]:
from collections import Counter
final = df[['task']]
for i in range(len(df)):
    counter = Counter(parse_answer(df.loc[i, f'response_{x}']) for x in range(8) if str(parse_answer(df.loc[i, f'response_{x}'])) not in ('...', 'None')).most_common(1)[0][0]
    final.loc[i, 'answer'] = '[' + str(counter) + ']'

/tmp/ipython-input-3146793227.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.loc[i, 'answer'] = '[' + str(counter) + ']'


In [ ]:
final.to_csv('final.csv', index=False)

# Авторы

Андрей Четверяков, Никита Бакутов